# Adaptable irrigation strategies 
## soil moisture targets

Previous work has examined the benifits of choosing the optimal irrigation stratgy for farm profits and water use efficiency.

Though as the season unfolds, farmers may want to adjust their irrigation strategy based on the climate conditions so far and their expectation of the weather for the rest of the season.


In [1]:
# !pip install aquacrop

In [2]:
from aquacrop.classes import *
from aquacrop.core import *
from aquacrop.lars import *

from scipy.optimize import differential_evolution
import numpy as np
from functools import partial
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from aquacropgym.envs import CropEnv, nebraska_maize_config, cordoba_cotton_config,california_tomato_config
from aquacropgym.utils import evaluate_agent,evaluate_agent_single_year
from aquacropgym.utils import calc_eto_faopm


In [4]:
class FixedThresholdAgent():
    def __init__(self,smts):
        self.smts=smts # gievn as fraction

    
    def compute_action(self,obs):
        "default observation set"

        return (self.smts*2)-1

## Champion - Maize

In [8]:

#CP  Lat.(deg)= 40.40  Long.(deg)=101.73  Elev.(m)=1072. 

gendf=calc_eto_faopm('data/CPWG.dat',1995,40.4,1072,True,["simyear","jday","minTemp","maxTemp","precip","rad"])


In [11]:
test_env_config=nebraska_maize_config.copy()

test_env_config['gendf']=gendf
test_env_config['evaluation_run']=True
test_env_config['normalize_obs']=False
# test_env_config['include_rain']=False
test_env_config['action_set']='smt4'
test_env_config['observation_set']='default'
# test_env_config['irrigation_cost']=1.
test_env_config['crop']='Cotton'
test_env_config['crop_price']=366.
# test_env_config['soil']='ClayLoam'



In [12]:
evaluate_agent(FixedThresholdAgent(np.array([0.69892958, 0.56825608, 0.35286359, 0.11124398])),CropEnv,test_env_config)

(-280.3196623997311, -279.6661787513153)

In [14]:
def optim_func(smt):
    train,test = evaluate_agent(FixedThresholdAgent(smt.reshape(-1)),CropEnv,test_env_config)
    
    return -train

In [15]:
max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]

res = differential_evolution(optim_func,bounds=bounds,disp=True,workers=8,seed=43,tol=0.05)

/home/mbyx3tk2/.local/lib/python3.8/site-packages/scipy/optimize/_differentialevolution.py:492: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  warnings.warn("differential_evolution: the 'workers' keyword has"


differential_evolution step 1: f(x)= 250.606
differential_evolution step 2: f(x)= 184.401
differential_evolution step 3: f(x)= 155.155
differential_evolution step 4: f(x)= 155.155
differential_evolution step 5: f(x)= 155.155
differential_evolution step 6: f(x)= 146.75
differential_evolution step 7: f(x)= 146.75
differential_evolution step 8: f(x)= 146.75
differential_evolution step 9: f(x)= 146.75
differential_evolution step 10: f(x)= 146.75
differential_evolution step 11: f(x)= 143.63
differential_evolution step 12: f(x)= 143.63


In [16]:
res

     fun: 143.63007343536125
 message: 'Optimization terminated successfully.'
    nfev: 785
     nit: 12
 success: True
       x: array([0.65195918, 0.28241524, 0.20799377, 0.17725579])

In [17]:
evaluate_agent(FixedThresholdAgent(res.x),CropEnv,test_env_config)

(-143.63007343536125, -151.97151695885836)

single year

In [ ]:
def optim_func_single_year(smt,sim_year=1):
    prof = evaluate_agent_single_year(FixedThresholdAgent(smt.reshape(-1)),CropEnv,test_env_config,sim_year=sim_year)
    
    return -prof

max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]



In [ ]:
from functools import partial

In [ ]:
optim_func_single_year(np.array([0.7]*4))

In [ ]:
years=[]
smts=[]
profs=[]
for y in range(1,101):
    func = partial(optim_func_single_year,sim_year=y)
    res = differential_evolution(func,bounds=bounds,disp=True,workers=8,seed=43)
    p = evaluate_agent_single_year(FixedThresholdAgent(res.x.reshape(-1)),CropEnv,test_env_config,y)
    years.append(y)
    smts.append(res.x)
    profs.append(p)
    result_df = pd.DataFrame([years,smts,profs]).T

    result_df.to_csv('potential/optim_smt_results_oldcrop.csv')

## cordoba - cotton

In [ ]:
#cd 37.31, 110
gendf=calc_eto_faopm('data/CDWG.dat',2018,37.31,110,True,["simyear","jday","minTemp","maxTemp","precip","rad"])

In [ ]:
test_env_config=cordoba_cotton_config.copy()

test_env_config['gendf']=gendf
test_env_config['evaluation_run']=True
test_env_config['normalize_obs']=False
test_env_config['action_set']='smt4'
test_env_config['observation_set']='default'
test_env_config['include_rain']=False


In [ ]:
evaluate_agent(FixedThresholdAgent(np.array([0.58709859, 0.66129679, 0.34608978, 0.10645481])),CropEnv,test_env_config)

In [ ]:
def optim_func(smt):
    train,test = evaluate_agent(FixedThresholdAgent(smt.reshape(-1)),CropEnv,test_env_config)
    
    return -train

In [ ]:
max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]

res = differential_evolution(optim_func,bounds=bounds,disp=True,workers=8,seed=43)

In [ ]:
res

In [ ]:
evaluate_agent(FixedThresholdAgent(np.array([0.66630478, 0.40196164, 0.2504096 , 0.26958299])),CropEnv,test_env_config)

## California - Tomato

In [5]:

#CL  37.49	-121.07	14.00 
gendf= calc_eto_faopm('data/CLWG.dat',1995,37.49,14.0,True,["simyear","jday","minTemp","maxTemp","precip","rad"])


In [6]:
test_env_config=california_tomato_config.copy()

test_env_config['gendf']=gendf
test_env_config['evaluation_run']=True
test_env_config['normalize_obs']=False
# test_env_config['include_rain']=False
test_env_config['action_set']='smt4'
test_env_config['observation_set']='default'
# test_env_config['irrigation_cost']=1.
# test_env_config['crop']='MaizeChampionGDD'
# test_env_config['soil']='ClayLoam'



In [7]:
wdf=gendf[gendf.simyear==4].drop('simyear',axis=1)

In [8]:
def run_model(smts,max_irr_season,year1,year2):
    """
    funciton to run model and return results for given set of soil moisture targets
    """

    maize = CropClass('Tomato',PlantingDate='05/1') # define crop
    loam = SoilClass('Loam') # define soil
    init_wc = InitWCClass(wc_type='Pct',value=[70]) # define initial soil water conditions

    irrmngt = IrrMngtClass(IrrMethod=1,SMT=smts,MaxIrrSeason=max_irr_season) # define irrigation management

    # create and run model
    model = AquaCropModel(f'{year1}/05/1',f'{year2}/12/31',wdf,loam,maize,
                          IrrMngt=irrmngt,InitWC=init_wc)
    model.initialize()
    model.step(till_termination=True)
    return model.Outputs.Final

run_model([70]*4,10_000,1995,1995)

,Season,Crop Type,Harvest Date (YYYY/MM/DD),Harvest Date (Step),Yield (tonne/ha),Seasonal irrigation (mm)
0,0,Tomato,1995-08-19,109,7.652106,756.363735


619.65

In [7]:
evaluate_agent(FixedThresholdAgent(np.array([0.69892958, 0.56825608, 0.35286359, 0.11124398])),CropEnv,test_env_config)

(270.9868053314722, 269.8517950217337)

In [8]:
evaluate_agent(FixedThresholdAgent(np.array([0.]*4)),CropEnv,test_env_config)

(4.046373497676493, 4.514327303782176)

In [9]:
def optim_func(smt):
    train,test = evaluate_agent(FixedThresholdAgent(smt.reshape(-1)),CropEnv,test_env_config)
    
    return -train

In [10]:
max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]

res = differential_evolution(optim_func,bounds=bounds,disp=True,workers=8,seed=43)

/home/mbyx3tk2/.local/lib/python3.8/site-packages/scipy/optimize/_differentialevolution.py:492: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  warnings.warn("differential_evolution: the 'workers' keyword has"


differential_evolution step 1: f(x)= -316.315
differential_evolution step 2: f(x)= -318.177
differential_evolution step 3: f(x)= -318.177
differential_evolution step 4: f(x)= -318.177
differential_evolution step 5: f(x)= -318.177
differential_evolution step 6: f(x)= -318.177
differential_evolution step 7: f(x)= -318.177
differential_evolution step 8: f(x)= -318.681
differential_evolution step 9: f(x)= -318.755


In [11]:
res

     fun: -318.7546385264889
 message: 'Optimization terminated successfully.'
    nfev: 605
     nit: 9
 success: True
       x: array([0.64511612, 0.65667727, 0.79650656, 0.04541987])

In [12]:
evaluate_agent(FixedThresholdAgent(res.x),CropEnv,test_env_config)

(318.7546385264889, 316.29092174163)

In [13]:
from aquacropgym.utils import get_mean_std

In [14]:
get_mean_std(res.x,CropEnv,test_env_config)

318.7546385264889
(1120, 15)


(array([1.52500000e+01, 6.50000000e+00, 3.15283656e-01, 6.03125000e+01,
        3.07615387e+02, 5.10523260e-01, 4.98117859e+02, 1.24298476e-01,
        6.86775160e+00, 1.21339521e+01, 4.01664978e+02, 1.87500000e-01,
        3.12500000e-01, 2.50000000e-01, 2.50000000e-01], dtype=float32),
 array([8.2120342e+00, 1.1180340e+00, 1.6626830e-01, 3.1970348e+01,
        2.0187964e+02, 2.8188160e-01, 4.2335272e+02, 5.5605030e-01,
        6.5448678e-01, 1.5673932e+01, 2.2597920e+02, 3.9031237e-01,
        4.6351239e-01, 4.3301269e-01, 4.3301269e-01], dtype=float32))